The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import time

%matplotlib inline

In [2]:
# Sample image read
# img = cv2.imread("./camera_cal/calibration1.jpg")
# plt.imshow(img)

In [3]:
# Camera Calibration
def camera_calibration(dir_calibration):
    n_row = 9
    n_col = 6
    n_ch  = 3

    # Generate world (3d) point set for the chekkerboard pattern
    objp = np.zeros((n_row*n_col,n_ch), np.float32)
    objp[:,:2] = np.mgrid[0:n_col, 0:n_row].T.reshape(-1,2)

    img_pt = [] #2d points, (x,y)
    obj_pt = [] #3d points, (x,y,z)

    #dir_calibration = "./camera_cal/"
    dir_dst_cameraCal = "./camera_cal_res/"
    for f_name in os.listdir(dir_calibration):
        img_file = os.path.join(dir_calibration, f_name)
        img = cv2.imread(img_file)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray_img, (n_row, n_col), None)
        if ret == True:
            img_pt.append(corners)
            obj_pt.append(objp)

            cv2.drawChessboardCorners(img, (n_row, n_col), corners, ret)
            #plt.imshow(img);plt.show()
            #plt.savefig(dir_dst_cameraCal+f_name)
    return (obj_pt, img_pt)
obj_pt, img_pt = camera_calibration("./camera_cal/")

In [4]:
#Distortion Correction - Test
dir_test = "./test_images/"
dir_out_dist_correction = "./out_dist_correction/"
def test_distortion_correction(dir_test, obj_pt, img_pt):
    for f_name in os.listdir(dir_test):
        img_bgr = cv2.imread(os.path.join(dir_test, f_name))
        img = cv2.cvtColor(img_bgr, cv2.COLOR_RGB2BGR)
        img_ht, img_wd, img_ch = img.shape
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_pt, img_pt, (img_ht, img_wd), None, None)
        undist_img = cv2.undistort(img, mtx, dist, None, mtx)
        plt.imshow(undist_img); 
        plt.savefig(dir_out_dist_correction+f_name)#plt.show()
#test_distortion_correction(dir_test, obj_pt, img_pt)

In [5]:
# Perspective transform
#dir_test = "./test_images/"
#warped_img_list = []
#for f_name in os.listdir(dir_test):
dir_out_persp_trans = "./out_persp_trans/"
def perspective_transform(img_bgr, f_name=None):
    #img_bgr = cv2.imread(os.path.join(dir_test, f_name))
    img = cv2.cvtColor(img_bgr, cv2.COLOR_RGB2BGR)
    img_ht, img_wd, img_ch = img.shape
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_pt, img_pt, (img_ht, img_wd), None, None)
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    # plt.imshow(undist_img); plt.show()
    # Points Order:  [top-left, top-right, btm-right, btm-left], (ht, wd)
    src_pt = np.float32([[490, 500], [800, 500],[1200, 680], [200, 680]])
    dst_pt = np.float32([[0,0], [img_wd, 0],[img_wd, img_ht], [0, img_ht]])
    perspective_M = cv2.getPerspectiveTransform(src_pt, dst_pt)
    warped = cv2.warpPerspective(undist_img, perspective_M, (img_wd, img_ht), flags=cv2.INTER_LINEAR)
    if f_name != None:
        plt.imshow(warped)
        plt.savefig(dir_out_persp_trans+f_name)
    #warped_img_list.append(warped)
    return undist_img, warped, perspective_M 
    #plt.imshow(warped); plt.show()

In [6]:
# Perspective transform


In [7]:
# Binary Threshold
dir_out_bin_img = "./out_bin_img/"
def binary_threshold(img, f_name=None):
    img_ht, img_wd, img_ch = img.shape
    #     # Extract Lane Lines
    #     hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    #     s_ch = hls[:,:,2]
    #     s_ch_thres = [140, 250]
    #     bin_img_y = np.zeros_like(s_ch)
    #     bin_img_y[(s_ch > s_ch_thres[0]) & (s_ch <= s_ch_thres[1])] = 1
    
    # White Line
    bin_s_ch = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    bin_l_ch = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,0]
    bin_b_ch = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,2]   

    s_thresh = [180, 255]
    s_bin = np.zeros_like(bin_s_ch)
    s_bin[(bin_s_ch >= s_thresh[0]) & (bin_s_ch <= s_thresh[1])] = 1
    
    b_thresh = [155, 200]
    b_bin = np.zeros_like(bin_b_ch)
    b_bin[(bin_b_ch >= b_thresh[0]) & (bin_b_ch <= b_thresh[1])] = 1
    
    l_thresh = [225, 255]
    l_bin = np.zeros_like(bin_l_ch)
    l_bin[(bin_l_ch >= l_thresh[0]) & (bin_l_ch <= l_thresh[1])] = 1
    
    bin_img = np.zeros_like(bin_s_ch)
    bin_img[(l_bin == 1) | (b_bin == 1) | (s_bin == 1)] = 1
    
    if f_name != None:
        plt.imshow(bin_img)
        plt.savefig(dir_out_bin_img+f_name)
    return bin_img
    #plt.imshow(bin_img)
    #bin_img_list.append(bin_img)
    #plt.show()


In [8]:
def hist(img):
    bottom_half = img[int(img.shape[0]/2):,:]
    histogram = np.sum(bottom_half, axis=0)
    return histogram

In [9]:
def fit_polynomial(histogram, binary_warped):
    # Hyperparams
    nwindows = 9
    margin = 100
    minpix = 5
    
    
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
    # Init window height
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Create an image to draw on and an image to show the selection window
    # out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high), (0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high), (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    #print(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    #print(righty, rightx)
    # Fit a second order polynomial to each
    isFailed = False
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    except:
        left_fit = None; right_fit = None
        isFailed = True

    return left_fit, right_fit, isFailed

In [10]:
def radius_curvature(binary_warped, left_fit, right_fit):
    isIrregularLanes = None
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Convert pixel values (y & x) to mtrs
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty) #y-Bottom
    y_top = np.min(ploty)
    
    # Fit polynomials for new space (for unwarped image)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # left & right radius of curvature
    left_curvature =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    right_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Vehicle centre calculation
    left_lane_bottom = (left_fit[0])*y_eval**2 + left_fit[0]*y_eval + left_fit[2]
    right_lane_bottom = (right_fit[0])*y_eval**2 + right_fit[0]*y_eval + right_fit[2]


    # Lane center as mid of left and right lane bottom                        
    lane_center = (left_lane_bottom + right_lane_bottom)/2
    center_image = 640  
    center = (lane_center - center_image)*xm_per_pix # Convert to mtrs
    position = "Left" if center < 0 else "Right"
    center = "Vehicle is "+ str(np.abs(round(center,3))) + " m " + position + " of center"

    return left_curvature, right_curvature, center

In [11]:
# #Test on images
# src_dir = "./camera_cal/"
# obj_pt, img_pt = camera_calibration(src_dir)
# dir_test = "./test_images/"
# dir_dst = "./out_test_images/"
# warped_img_list = []
# for f_name in os.listdir(dir_test):
#     img_bgr = cv2.imread(os.path.join(dir_test, f_name))
#     # Perspective transform
#     undist_img, warped, perspective_M  = perspective_transform(img_bgr, f_name)
#     # Binary threshold
#     bin_image = binary_threshold(warped, f_name)
#     # Histogram
#     histogram = hist(bin_image)
#     left_fit, right_fit,left_lane_inds, right_lane_inds, nonzerox, nonzeroy, isFailed = fit_polynomial(histogram, bin_image)
#     # Calculate the radius of curvature in pixels for both lane lines
#     left_curv, right_curv, center, isIrregularLanes = radius_curvature(bin_image, left_fit, right_fit)

#     res_frame_bgr = overlay_frame(undist_img, bin_image, left_fit, right_fit, perspective_M, left_curv, right_curv, center)
#     res_frame = cv2.cvtColor(res_frame_bgr, cv2.COLOR_BGR2RGB)
#     plt.imshow(res_frame)
#     #plt.show()
#     plt.savefig(dir_dst+f_name)
#     #print(os.path.join(dir_test, f_name))
#     #print(left_curvature, right_curvature, center)


In [18]:
def overlay_frame(undist_img, warped, pts, perspective_M, left_curv, right_curv, center, frame_counter):
    # Few segments of code customized from Quiz on Lesson 9, Module 4
    
    # Create a dummy zeros only array eq. to warped image
    warped_zeros = np.zeros_like(warped).astype(np.uint8)
    # Create an output image to draw overlay and put on over display frame(res_frame)
    res_frame_mask = np.dstack((warped_zeros, warped_zeros, warped_zeros))

    # https://stackoverflow.com/a/18817152
    # points = np.array([[1, 2], [3, 4]]) # cv2.polylines(img, np.int32([points]), 1, (255,255,255))
    cv2.polylines(res_frame_mask, np.int32([pts]), False, (0,0,255), thickness = 60)
    cv2.fillPoly(res_frame_mask, np.int32([pts]), (0,255, 0))
    
    # Warped to actual frame matrix
    actual_M = np.linalg.inv( perspective_M)
    # reframe mask to be fit over res_frame
    rewarped = cv2.warpPerspective(res_frame_mask, actual_M, (res_frame_mask.shape[1], res_frame_mask.shape[0]))
    # 
    res_frame_bgr = cv2.addWeighted(undist_img, 1, rewarped, 0.3, 0)
    res_frame = cv2.cvtColor(res_frame_bgr, cv2.COLOR_BGR2RGB)
    
    text_str = "Radius of Curvature (m) is " + str(int(left_curv) if (left_curv < right_curv) else int(right_curv))
    cv2.putText(res_frame, text_str, (50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    text_str = center
    cv2.putText(res_frame, text_str, (50, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
#     text_str = "Frame No: " + str(frame_counter)
#     cv2.putText(res_frame, text_str, (50, 140), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 100, 100), thickness=2)
    
#     text_str = "Left: " + str(data_tup[0]) + "  Right: " + str(data_tup[1]) + " <<>> " + str(data_tup[2])
#     cv2.putText(res_frame, text_str, (50, 180), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 100, 100), thickness=2)
#     save_dir = "./out_rev3_dump/dmp_" + str(frame_counter) + ".jpg"
#     result_img_bgr = cv2.cvtColor(res_frame, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(save_dir, result_img_bgr)
    
    return res_frame

In [19]:
def overlay_line_gen(bin_image, left_fit, right_fit):

    ploty = np.linspace(0, bin_image.shape[0]-1, bin_image.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    
    isIrregularLanes = False
    
    right_lane_btm =  pts_right[0][-1][0]
    left_lane_btm  = pts_left[0][0][0]
    
    left_lane_top = pts_left[0][-1][0]
    right_lane_top = pts_right[0][0][0]
    
#     print("Btm: ", left_lane_btm, right_lane_btm)
#     print("Top: ", left_lane_top, right_lane_top)
    

    if ((right_lane_btm - left_lane_btm) < 500) or((right_lane_btm - left_lane_btm) > 1200 ) :
        isIrregularLanes = True
    if (right_lane_btm < 500 ) or (left_lane_btm > 1000) :
        isIrregularLanes = True

    if ((right_lane_top - left_lane_top) < 200) or((right_lane_top - left_lane_top) > 1200 ) :
        isIrregularLanes = True
    if (right_lane_top < 900 ) or (left_lane_top > 200) :
        isIrregularLanes = True
        
    pts = np.hstack((pts_left, pts_right))
    
    data_tup = [left_lane_top, right_lane_top, isIrregularLanes]
    return pts, isIrregularLanes, data_tup

In [23]:
#Video Pipeline
prev_frame_data = None #Previous frame data to draw lane 
prev_frame_info = None #Previous frame data to overlay Radius of curv & Center
frame_counter = 0
noPrevData = True #Check if we have previous data
def process_image(image):
    global prev_frame_info
    global prev_frame_data
    global frame_counter
    global noPrevData
    # Control priniting of Radius of curvature and Centre every third frame only
    if (frame_counter % 1 == 0) or (frame_counter == 0):
        writeFlag = True
    else:
        writeFlag = False
    # Perspective transform
    undist_img, warped, perspective_M = perspective_transform(image)
    # Binary threshold
    bin_image = binary_threshold(warped)
    # Histogram
    histogram = hist(bin_image)
    left_fit, right_fit, isFailed= fit_polynomial(histogram, bin_image)
    if not isFailed:
        pts, isIrregularLanes, data_tup = overlay_line_gen(bin_image, left_fit, right_fit)

    # If isFailed flag is true, indicates failed to find points/generate line polynomial, use previous frame information
    if ( (isFailed ) and (noPrevData == True) ):
        result_img = image
    
    elif ((not isFailed) or (noPrevData == True) ):
        #print("Here1 ", writeFlag, frame_counter, noPrevData , end="")
        #if frame_counter == 0:
        #    result_img = overlay_frame(undist_img, bin_image, left_fit, right_fit, perspective_M, left_curvature, right_curvature, center)
        if ( (writeFlag == True) or (frame_counter == 0) or (noPrevData == True)):
            #print("Here2 ", end="")
            left_curvature, right_curvature, center = radius_curvature(bin_image, left_fit, right_fit)

            if ((noPrevData == True) and (isIrregularLanes == True)):
                #print("Here-NotStarted", frame_counter)
                result_img = image
                noPrevData = True
            elif (noPrevData == False) and isIrregularLanes == True:
                #print("Herex2", frame_counter)
                pts, perspective_M = prev_frame_data
                left_curvature, right_curvature, center = prev_frame_info
                result_img = overlay_frame(undist_img, bin_image, pts, perspective_M, left_curvature, right_curvature, center, frame_counter)
            elif isIrregularLanes == False:
                #print("Herex3", frame_counter)
                result_img = overlay_frame(undist_img, bin_image, pts, perspective_M, left_curvature, right_curvature, center, frame_counter)
                prev_frame_data = [pts, perspective_M]
                prev_frame_info = [left_curvature, right_curvature, center]
                noPrevData = False
        else:
            #print("Herex4 ", end="")
            if isIrregularLanes == False:
                #print("Here _Q_1 ", end="")
                left_curvature, right_curvature, center = prev_frame_info
                result_img = overlay_frame(undist_img, bin_image, pts, perspective_M, left_curvature, right_curvature, center, frame_counter)
            elif isIrregularLanes ==  True:
                #print("Here _Q_1 ", end="")
                pts, perspective_M = prev_frame_data
                left_curvature, right_curvature, center = prev_frame_info
                result_img = overlay_frame(undist_img, bin_image, pts, perspective_M, left_curvature, right_curvature, center, frame_counter)    
    else:
        #print("Here _Z_ ", end="")
        if ((frame_counter == 0) or (noPrevData==True) ):
            #print("Here _Z_1 ", end="")
            result_img = image
            noPrevData = True
        else:
            #print("Here _Z_2 ", end="")
            pts, perspective_M = prev_frame_data
            left_curvature, right_curvature, center = prev_frame_info
            result_img = overlay_frame(undist_img, bin_image, pts, perspective_M, left_curvature, right_curvature, center, frame_counter)
    frame_counter += 1
    # Calculate the radius of curvature in pixels for both lane lines
    #left_curvature, right_curvature, center = radius_curvature(bin_image, left_fit, right_fit)    
    #result_img = overlay_frame(undist_img, bin_image, left_fit, right_fit, perspective_M, left_curvature, right_curvature, center)
    #prev_frame_info = [left_fit, right_fit, perspective_M, left_curvature, right_curvature, center]
    #print(result_img.shape)
    return result_img

In [24]:
## Video Processing
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'output_rev2.mp4'

clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(39, 43)
#clip1 = VideoFileClip("project_video.mp4").subclip(40, 43)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:

# #Test on images
# src_dir = "./camera_cal/"
# obj_pt, img_pt = camera_calibration(src_dir)
# dir_test = "./test_images/"
# dir_dst = "./out_test_images/"
# warped_img_list = []
# for f_name in os.listdir(dir_test):
#     img_bgr = cv2.imread(os.path.join(dir_test, f_name))
#     # Perspective transform
#     undist_img, warped, perspective_M  = perspective_transform(img_bgr, f_name)
#     # Binary threshold
#     #bin_image = binary_threshold(warped, f_name)
#     img = warped

#     # White Line
#     bin_s_ch = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
#     bin_l_ch = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,0]
#     bin_b_ch = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,2]   

#     s_thresh = [180, 255]
#     s_bin = np.zeros_like(bin_s_ch)
#     s_bin[(bin_s_ch >= s_thresh[0]) & (bin_s_ch <= s_thresh[1])] = 1
    
#     b_thresh = [155, 200]
#     b_bin = np.zeros_like(bin_b_ch)
#     b_bin[(bin_b_ch >= b_thresh[0]) & (bin_b_ch <= b_thresh[1])] = 1
    
#     l_thresh = [225, 255]
#     l_bin = np.zeros_like(bin_l_ch)
#     l_bin[(bin_l_ch >= l_thresh[0]) & (bin_l_ch <= l_thresh[1])] = 1
    
#     bin_img = np.zeros_like(bin_s_ch)
#     bin_img[(l_bin == 1) | (b_bin == 1) | (s_bin == 1)] = 1    
#     binary_warped = bin_img 
    
# #     plt.imshow(img); plt.show()
# #     plt.imshow(bin_img); plt.show()
# #     #plt.imshow(s_binary); plt.show()

#     print("_END_----")

